In [102]:
import numpy as np
import pandas as pd
import math
import random

# set the seed for reproducibility
random.seed(123)

# particle specification, will be made variable later
P0 = 'p'; E0 = 800
P1 = 'a'; E1 = 0

# number of events per simulation
nEvents = 100

# number of simulations
nSim = 10

# standard variation of parameters
sigma = 0.1

# determine if parameter file should be overwritten or appended
overwrite = True

# name of simulations
simName = 'April4-'

In [108]:
# read in the default parameters from GADGET.sim.par
# and store them in a dictionary
# key: parameter name

param_dict = {}

with open('templates/GADGET.sim.par') as f:
    lines = f.readlines()
for i in range(len(lines)):
    # if line is commented, skip
    if lines[i][0] == '#':
        continue

    seg0 = lines[i].split(':')[0]
    seg1 = lines[i].split(':')[-1]
    if 'Int_t' in seg1 or 'Double_t' in seg1:
        # extract the number from seg1
        for j in seg1.split('#')[0].split(' '):
            if j != '':
                try:
                    float(j)
                    if 'Int_t' in seg1:
                        param_dict[seg0] = [int(j), 0]
                    else:
                        param_dict[seg0] = [float(j)]
                        # determine precision of parameter
                        if '.' in j:
                            param_dict[seg0].append(len(j.split('.')[1]))
                        else:
                            param_dict[seg0].append(1)

                except:
                    pass

# Add additional parameters by hand
param_dict['E0'] = [E0, 0]
param_dict['E1'] = [E1, 0]

param_dict['Xb'] = [2, 0]
param_dict['Yb'] = [2, 0]
param_dict['Zb1'] = [10, 0]
param_dict['Zb2'] = [40, 0]

param_dict['Threshold'] = [20, 0]

In [104]:
def param_variation(val0, var, nSim=10, var_type='uniform', fraction = False, decimal=0):
    # val0: the base value
    # var: the variation
    # var_type: the type of distribution to use
    # fraction: whether the variation is a fraction of the base value
    # decimal: the number of decimal places to round to

    if fraction == True:
        var *= val0

    if var_type == 'uniform':
        val = np.random.uniform(val0-var, val0+var, nSim)
    elif var_type == 'normal':
        val = np.random.normal(val0, var, nSim)
    elif var_type == 'triangular':
        val = np.random.triangular(val0-var, val0, val0+var, nSim)
    else:
        raise ValueError('var_type not valid')
    
    val = np.round(val, decimal)
    if decimal == 0:
        val = val.astype(int)
    return val

In [105]:
if overwrite:
    param_df = pd.DataFrame(columns=['Sim', 'Status', 'N', 'P0', 'E0', 'P1', 'E1', 'Xb', 'Yb', 'Zb1', 'Zb2', 'Threshold'])
else:
    param_df = pd.read_csv('parameters.csv')

In [106]:
for i in range(nSim):
    # create a new row in the dataframe
    param_df.loc[i] = [simName + str(i+1), 0, nEvents, P0, E0, P1, E1, 2, 2, 10, 40, 20]

    # vary the parameters
    for key in param_dict:
        if key in ['Sim', 'Status', 'N', 'P0', 'E0', 'P1', 'E1', 'Xb', 'Yb', 'Zb1', 'Zb2', 'Threshold']:
            continue
        val = param_variation(param_dict[key][0], sigma*param_dict[key][0], 1, 'normal', False, param_dict[key][1])
        param_df.loc[i, key] = val[0]

,Sim,Status,N,P0,E0,P1,E1,Xb,Yb,Zb1,Zb2,Threshold
